In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import ray
from ray.rllib.agents import ppo

from mdps.cliff import ContextualCliff
from utils.distributions import ConstantDistribution, ParticleDistribution, UniformDistribution

In [ ]:
# some utils used later

def prepare_animation(bar_container_, context_history_):
    """Used for animated plotting sampling precedure"""
    def animate(frame_number, data=context_history_):
        # simulate new data coming in
        x = data[frame_number][:, 1]
        n, _ = np.histogram(x, HIST_BINS)
        plt.title(f'{frame_number} posterior mean: {round(x.mean(), 5)}')
        for count, rect in zip(n, bar_container_.patches):
            rect.set_height(count)
        return bar_container_.patches

    return animate


# collect expert rollout

def get_rollouts(solver_, config):
    """Generate rollouts from a given solver and MDP(c)"""
    env_ = ContextualCliff(config=config)
    done_ = False
    obs_ = env_.reset()
    # run until episode ends
    gt_obs_arr_ = None
    gt_act_arr_ = None
    while not done_:
        action_ = solver_.compute_single_action(obs_)
        obs_, _, done_, _ = env_.step(action_)
        if gt_obs_arr_ is None:
            gt_obs_arr_ = obs_
            gt_act_arr_ = [action_]
        else:
            gt_obs_arr_ = np.vstack((gt_obs_arr_, obs_))
            gt_act_arr_ += [action_]

    gt_act_arr_ = np.array(gt_act_arr_)
    return gt_obs_arr_, gt_act_arr_

def plot_rollouts(gt_obs_arr_, gt_act_arr_):
    """Plot generated rollouts"""
    fig_, ax_1 = plt.subplots()
    fig_.set_size_inches(10, 6, forward=True)

    x = np.arange(start=0, stop=gt_obs_arr_.shape[0])
    ax_2 = ax_1.twinx()
    ax_1.plot(x, gt_obs_arr_[:, 0], 'r-')
    ax_2.plot(x, gt_act_arr_, 'b-', alpha=0.3)

    ax_1.set_xlabel('time step')
    ax_1.set_ylabel('Position (x)', color='r')
    ax_2.set_ylabel('Action', color='b')
    plt.title('sample observations and actions')
    plt.show()

# Contextual "cliff"

This notebook studies using particle filtering to estimate the context parameters of a standard `Cliff` environment in `mdps.cliff`.

The `Cliff` environment has a cart that chooses either to go left or right for a `step_size`. It starts at the `mid_point` of the `right_end` and the `left_end`, and it receives a reward proportional to the power (default set to 2) of its current location, `x`. If it "falls off the cliff" from the left or right end, however, it receives a highly negative reward and the episode ends. Also, there are noise and drift terms, and the state-transition equation of the position `x` is (where `action` is 0 or 1 for moving left or right):
\
$x_{t+1} = N(0, \text{noise}) - \text{drift} * x + 2 * (\text{action}-0.5) * \text{stepsize}$

Thus, the goal of the cart is to keep its position `x` as large as possible (i.e. close to the larger end) and in the meantime keep a distance to that end to prevent accidentally falling off due to the transition noise.

The context params are: `(left_bound, right_bound, pow, step_size, noise, drift)`.

# Set target

We first create a target config, $c$. This will be the $MDP(c)$ the expert uses to generate the observational data.

In [ ]:
# true (expert) context: (left_bound, right_bound, pow, step_size, noise, drift) =
#                         [0.0, 3.0, 2.0, 0.05, 0.05, 0.0]
c = {'context_distribution':
         ConstantDistribution(dim=6,
                              constant_vector=np.array([0.0, 2, 2, 0.05, 0.05, 0.0]))}

First, train an expert that masters this context.

In [ ]:
ray.shutdown()
ray.init()
expert = ppo.PPOTrainer(env=ContextualCliff, config={
    "env_config": c,
    "framework": "torch",  # config to pass to env class
})

rews = []
for eps in range(25):
    res = expert.train()
    if eps % 5 == 0:
        print(eps, res['episode_reward_mean'])
    rews += [res['episode_reward_mean']]

In [ ]:
# collect expert rollout
gt_obs_arr, gt_act_arr = get_rollouts(expert, config=c)

In [ ]:
plot_rollouts(gt_obs_arr, gt_act_arr)

## Exact context

We first train a RL solver only on the correct context for particle filtering. We have already done this so we can directly use the expert solver.

In [ ]:
# true (expert) context: (left_bound, right_bound, pow, step_size, noise, drift) =
#                       [0.0, 3.0, 2.0, 0.025, 0.05, 0.0]

N = 2000
T = 100
left_bound = np.ones((N,)) * 0.0
pow = np.ones((N,)) * 2
drift = np.random.normal(loc=0.00, scale=0.001, size=(N,))
step_size = np.ones((N,)) * 0.025

In [ ]:
def filter_context(solver_,
                   context_distribution_,
                   gt_obs_arr_,
                   T_,
                   N_
                   ):
    state_arr_ = np.zeros((N_,))
    action_arr_ = np.zeros((N_,))
    context_history_ = []
    for t_ in range(T_):
        # we only use the first 5 steps of the cartpole steps to reduce effect of different episode lengths
        qs_ = np.zeros((N_,))
        for n_ in range(N_):
            context_ = context_distribution_.particles[n_]
            c_local_ = {'context_distribution':
                           ConstantDistribution(dim=6,
                                                constant_vector=context_)}
            env_ = ContextualCliff(config=c_local_)
            obs_ = env_.reset()
            if t_ > 0:
                env_.mdp.x = state_arr_[n_]
                obs_ = np.concatenate((np.array([env_.mdp.x]), context_), axis=0).flatten()
            action_ = solver_.compute_single_action(obs_)
            obs_, _, done_, _ = env_.step(action_)
            # estimate likelihood if r >= 1
            if t_ >= 1:
                q = env_.likelihood(gt_obs_arr_[t_ - 1], action_arr_[n_], obs_)
                qs_[n_] = q
            state_arr_[n_] = np.copy(env_.mdp.x)
            action_arr_[n_] = action_
        if t_ >= 1:
            # truncated importance sampling; [https://arxiv.org/pdf/1905.09800.pdf]
            qs_ = np.clip(qs_, 0, np.percentile(qs_, 90))
            qs_ = qs_ / qs_.sum()
            resample_index_ = context_distribution_.resample_particles_from_probability(p=qs_)
            p_temp_ = context_distribution_.particles
            p_noise_ = np.random.normal(loc=0, scale=p_temp_.std(axis=0), size=p_temp_.shape) * 0.05
            context_distribution_.particles += p_noise_
            state_arr_ = state_arr_[resample_index_]
            action_arr_ = action_arr_[resample_index_]
        if t_ % 10 == 0:
            print("round", t_, "posterior mean", context_distribution_.particles[:, 1].mean())
        context_history_ += [context_distribution_.particles.copy()]
    return context_history_

In [ ]:
right_bound = np.random.normal(loc=2.5, scale=0.5, size=(N,))
noise = np.ones((N,)) * 0.05

context_particles = np.abs(np.vstack((left_bound, right_bound, pow, step_size, noise, drift)).T)
context_distribution = ParticleDistribution(dim=6, particles=context_particles, n_particles=N)

context_history_exact = filter_context(expert,
                                   context_distribution,
                                   gt_obs_arr,
                                   T,
                                   N
                                   )

In [ ]:
HIST_BINS = np.linspace(1, 3, 120)

fig, ax = plt.subplots()
plt.axvline(x=2.0, alpha=0.3, color='black', linestyle='--')
plt.legend(['ground truth: 2.0'])
_, _, bar_container = ax.hist(context_history_exact[0][:, 1], HIST_BINS, lw=1,
                              ec="cyan", fc="blue", alpha=0.5)
ax.set_ylim(top=N / 4)  # set safe limit to ensure that all data is visible.

ani = animation.FuncAnimation(fig, prepare_animation(bar_container, context_history_exact),
                              len(context_history_exact),
                              repeat=True, blit=False, interval=100, repeat_delay=500)
prior_mean = 2.5
posterior_mean = round(context_history_exact[-1][:, 1].mean(), 3)
ani.save(f'prior_{prior_mean}_posterior{posterior_mean}.mp4', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots()
HIST_BINS = np.linspace(1, 4, 80)
ax.hist(context_history_exact[0][:, 1], HIST_BINS, lw=1,
        ec="blue", fc="blue", alpha=0.5)
ax.hist(context_history_exact[-1][:, 1], HIST_BINS, lw=1,
        ec="red", fc="red", alpha=0.5)
plt.axvline(x=2, alpha=0.3, color='black', linestyle='--')
plt.legend(['ground truth: 2.0', 'prior', 'posterior'])
ax.set_ylim(top=N / 5)
fig.set_size_inches(12, 8)
plt.title('solver trained in exactly specified context')
plt.show()

In [ ]:
# We reduce the right end from 2.0 to 1.5 and keep otherwise the same.
c_mis = {'context_distribution':
             ConstantDistribution(dim=5,
                                  constant_vector=np.array([0.0, 1.5, 2.0, 0.025, 0.05, 0.0]))}

ray.shutdown()
ray.init()
mis_solver = ppo.PPOTrainer(env=ContextualCliff, config={
    "env_config": c_mis,
    "framework": "torch",  # config to pass to env class
})

for eps in range(25):
    res = mis_solver.train()
    if eps % 5 == 0:
        print(eps, res['episode_reward_mean'])

In [ ]:
mis_obs_arr, mis_act_arr = get_rollouts(mis_solver, config=c)
plot_rollouts(mis_obs_arr, mis_act_arr)

In [ ]:
# true (expert) context: (left_bound, right_bound, pow, step_size, noise, drift) =
#                       [0.0, 3.0, 2.0, 0.025, 0.05, 0.0]

right_bound = np.random.normal(loc=2.5, scale=0.5, size=(N,))
noise = np.ones((N,)) * 0.05

context_particles = np.abs(np.vstack((left_bound, right_bound, pow, step_size, noise, drift)).T)
context_distribution = ParticleDistribution(dim=6, particles=context_particles, n_particles=N)

context_history_mis = filter_context(mis_solver,
                                       context_distribution,
                                       gt_obs_arr,
                                       T,
                                       N
                                       )

In [ ]:
fig, ax = plt.subplots()
HIST_BINS = np.linspace(1, 5, 80)
ax.hist(context_history_mis[0][:, 1], HIST_BINS, lw=1,
        ec="blue", fc="blue", alpha=0.5)
ax.hist(context_history_mis[-1][:, 1], HIST_BINS, lw=1,
        ec="red", fc="red", alpha=0.5)
plt.axvline(x=2, alpha=0.3, color='black', linestyle='--')
plt.legend(['ground truth: 2.0', 'prior', 'posterior'])
ax.set_ylim(top=N / 5)
fig.set_size_inches(12, 8)
plt.title('solver trained in mis-specified context')
plt.show()

# Uniformly sampled context

We uniformly sample contexts during training, a common way to perform domain randomization.

In [ ]:
c_uniform = {'context_distribution':
             UniformDistribution(dim=6,
                              lower_bound_vector=np.array([0.0, 1.0, 2, 0.05, 0.05, 0.0]),
                              upper_bound_vector=np.array([0.0, 3.0, 2, 0.05, 0.05, 0.0]))}

In [ ]:
ray.shutdown()
ray.init()
uniform_solver = ppo.PPOTrainer(env=ContextualCliff, config={
                                                    "env_config": c_uniform,
                                                    "framework": "torch",  # config to pass to env class
                                                })

rews = []
for eps in range(25):
    res = uniform_solver.train()
    if eps % 5 == 0:
        print(eps, res['episode_reward_mean'])
    rews += [res['episode_reward_mean']]

In [ ]:
plot_rollouts(*get_rollouts(uniform_solver,
                           config={'context_distribution':
                                    ConstantDistribution(dim=5,
                                    constant_vector=np.array([0.0, 1.5, 2.0, 0.05, 0.05, 0.0]))}))

In [ ]:
plot_rollouts(*get_rollouts(uniform_solver,
                           config={'context_distribution':
                                    ConstantDistribution(dim=5,
                                    constant_vector=np.array([0.0, 2.5, 2.0, 0.05, 0.05, 0.0]))}))

In [ ]:
right_bound = np.random.normal(loc=2.5, scale=0.5, size=(N,))
noise = np.ones((N,)) * 0.05

context_particles = np.abs(np.vstack((left_bound, right_bound, pow, step_size, noise, drift)).T)
context_distribution = ParticleDistribution(dim=6, particles=context_particles, n_particles=N)

context_history_uniform = filter_context(uniform_solver,
                                           context_distribution,
                                           gt_obs_arr,
                                           T,
                                           N
                                           )

In [ ]:
fig, ax = plt.subplots()
HIST_BINS = np.linspace(1, 4, 80)
ax.hist(context_history_uniform[0][:, 1], HIST_BINS, lw=1,
        ec="blue", fc="blue", alpha=0.5)
ax.hist(context_history_uniform[-1][:, 1], HIST_BINS, lw=1,
        ec="red", fc="red", alpha=0.5)
plt.axvline(x=2, alpha=0.3, color='black', linestyle='--')
plt.legend(['ground truth: 2.0', 'prior', 'posterior'])
ax.set_ylim(top=N / 5)
fig.set_size_inches(12, 8)
plt.title('solver trained in uniformly sampled context')
plt.show()

# Importance sampling for context

We do training - PF filtering update - training loop.

In [35]:
rews = []
c_mut = ConstantDistribution(dim=6,
                                  constant_vector=np.array([0.0, 0.5, 2, 0.05, 0.05, 0.0]),
                                     )
mut_context_dist = {'context_distribution': c_mut}
ray.shutdown()
ray.init()
imp_solver = ppo.PPOTrainer(env=ContextualCliff, config={
                                                    "env_config": mut_context_dist,
                                                    "framework": "torch",  # config to pass to env class
                                                })
for eps in range(25):
    res = imp_solver.train()
    print(eps, res['episode_reward_mean'])
    rews += [res['episode_reward_mean']]
    c_mut.update({'constant_vector': np.array([0.0, 2.0+eps/5, 2, 0.05, 0.05, 0.0])})
    imp_solver.config['env_config']['context_distribution'] = c_mut
    print(imp_solver.config['env_config']['context_distribution'].constant)

2022-02-14 15:57:05,331	INFO services.py:1340 -- View the Ray dashboard at http://127.0.0.1:8265
2022-02-14 15:57:07,635	WARNING util.py:57 -- Install gputil for GPU system monitoring.


0 -99.76200537262714
[0.   2.   2.   0.05 0.05 0.  ]
1 -99.41105112858652
[0.   2.2  2.   0.05 0.05 0.  ]
2 -98.18317576491216
[0.   2.4  2.   0.05 0.05 0.  ]
3 -87.46754835771316
[0.   2.6  2.   0.05 0.05 0.  ]
4 -77.2837027627055
[0.   2.8  2.   0.05 0.05 0.  ]
5 -75.5385437791193
[0.   3.   2.   0.05 0.05 0.  ]
6 -69.10061017438866
[0.   3.2  2.   0.05 0.05 0.  ]
7 -58.6658449707593
[0.   3.4  2.   0.05 0.05 0.  ]
8 -37.242462217478895
[0.   3.6  2.   0.05 0.05 0.  ]
9 -31.784324284554405
[0.   3.8  2.   0.05 0.05 0.  ]
10 -28.64796325083168
[0.   4.   2.   0.05 0.05 0.  ]
11 -24.61236093808337
[0.   4.2  2.   0.05 0.05 0.  ]
12 -23.401666954561087
[0.   4.4  2.   0.05 0.05 0.  ]
13 -13.558363910959429
[0.   4.6  2.   0.05 0.05 0.  ]
14 -11.174483153082623
[0.   4.8  2.   0.05 0.05 0.  ]
15 -25.60910890901097
[0.   5.   2.   0.05 0.05 0.  ]
16 -30.443297434663712
[0.   5.2  2.   0.05 0.05 0.  ]
17 -28.04832344331563
[0.   5.4  2.   0.05 0.05 0.  ]
18 -23.88609293371605
[0.   5.6  2.

In [31]:
imp_solver.__dict__

{'_env_id': 'ContextualCliff',
 'env_creator': <function ray.rllib.agents.trainer.Trainer._register_if_needed.<locals>.<lambda>(cfg)>,
 'local_replay_buffer': None,
 '_experiment_id': 'abc276d335b34e37a74373ad532c6af0',
 'config': {'num_workers': 2,
  'num_envs_per_worker': 1,
  'create_env_on_driver': False,
  'rollout_fragment_length': 200,
  'batch_mode': 'truncate_episodes',
  'gamma': 0.99,
  'lr': 5e-05,
  'train_batch_size': 4000,
  'model': {'_use_default_native_models': False,
   '_disable_preprocessor_api': False,
   'fcnet_hiddens': [256, 256],
   'fcnet_activation': 'tanh',
   'conv_filters': None,
   'conv_activation': 'relu',
   'post_fcnet_hiddens': [],
   'post_fcnet_activation': 'relu',
   'free_log_std': False,
   'no_final_linear': False,
   'vf_share_layers': False,
   'use_lstm': False,
   'max_seq_len': 20,
   'lstm_cell_size': 256,
   'lstm_use_prev_action': False,
   'lstm_use_prev_reward': False,
   '_time_major': False,
   'use_attention': False,
   'attention